# RNN for An Alignment Cost-Based Classification of Log Traces

# 0. Fitness

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd 
import numpy as np
np.random.seed(0)
import time

from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation
#from keras.layers.embeddings import Embedding
from tensorflow.python.keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Bidirectional

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from statistics import mean

In [2]:
def fitness(packageForFitness, minRunLength):
    '''
    This function computes the fitness for all the sequences. 
    @sequences: the sequences of words
    @costs: the real alignment cost
    @minRunLength: the minimal run in the alignment dataset
    '''
    sumTraceFitness = 0
    totTraces = 0 
    for i in packageForFitness.index:
        sumTraceFitness += (1 - (packageForFitness.realCosts[i] / ( packageForFitness.lengths[i]  + minRunLength )))#*packageForFitness.freqs[i]
        #totTraces += packageForFitness.freqs[i]
        totTraces += 1
    return sumTraceFitness / totTraces

def LB_fitness(packageForFitness, minRunLength, m_AC, indices):
    '''
    This function computes lower bound of the fitness given in the paper. 
    @sequences: the sequences of words
    @minRunLength: the minimal run in the alignment dataset
    @m_AC: needed for the lowerbound formula
    @indices: if we compute the lower bound, then we don't iterate on all the traces but only the positives. 
    '''
    sumTraceFitness = 0
    totTraces = 0 
    for i in indices:
        sumTraceFitness += (1 - ((m_AC) / ( packageForFitness.lengths[i] + minRunLength )))#*packageForFitness.freqs[i]
    for i in packageForFitness.index:
        #totTraces += packageForFitness.freqs[i]
        totTraces += 1
    return sumTraceFitness / totTraces

# 1. Preprocessing the data

In [3]:
# must be a global variable for fake data
word_to_index = {}
max_len = 0

In [4]:
def cleanDataForRNN(dataFile,m_AC, word_to_index_already_exists=None):
    '''
    Reads the file (1), specifies the target classes (2) and prepare the sequences of activities (3). 
    Finally, its prepare some variables `sequences`, `costs` and `minRunLenght` for computing fitness. 
    @dataFile: (String) filename of the alignment dataset
    @m_AC: (int) maximal alignment cost classifier
    '''
    # ---- (1) read the file 
    data = pd.read_csv(dataFile,sep = ";", 
                   names = ["traces","tracesWithMoves","runs","runsWithMoves","costs","frequencies"])
    
    # ---- (2) create the positive and negative target depending on the m_AC parameter
    # alignment cost which interests us is greater than 10000 (other costs are just silent moves)
    # set the fitting to tmp_pos to set them latter to 1
    y = (((data["costs"] / 10000).astype(int)) / (m_AC+1)).astype(int)
    max_y = y.max()
    y = y.replace(0,"tmp_pos")
    y = y.replace(range(1,max_y + 1), 0)
    y = y.replace("tmp_pos",1)

    # two columns are required for a binary classification in a RNN
    y = np.eye(2)[y.to_numpy().reshape(-1)]
    
    # ---- (3) prepare the sequences of activities 
    traces_to_matrix = data.traces.str.split(":::",expand=True,)
    if word_to_index_already_exists== None:
        global max_len
        number_of_traces, max_len = traces_to_matrix.shape
    
    # transform the matrix to a serie
    traces_to_serie = pd.concat([traces_to_matrix[i] for i in range(0,traces_to_matrix.shape[1])], axis=0, 
                                          ignore_index=True, sort=False)
    # from the serie, it's easy to get unique words
    index_to_word = list(filter(None,(traces_to_serie).unique()))
    number_of_activities = len(index_to_word)

    # a simple dictionary 
    if word_to_index_already_exists==None:
        global word_to_index 
        word_to_index = { index_to_word[i]:i+1 for i in range(0, len(index_to_word) ) }
    
    # loop over traces 
    x = np.zeros((data.traces.shape[0],max_len))   
    for i in range(data.traces.shape[0]):
        # Convert the ith training trace and split is into activities. 
        trace = data.traces[i].split(":::")[:-1]
        # for every activity name in ith trace, jth is its index /!\ 
        j = 0
        # Loop over the activities 
        for w in trace:
            # Set the (i,j)th entry of X_trains to the index of the correct word.
            x[i, j] = word_to_index[w]
            # Increment j to j + 1
            j += 1   
    
    # for fitness computation
    minLengthRun= len(data.runs.str.split(":::").min())
    lengths = data.traces.str.split(":::",expand=False,).str.len()-1
    realCosts = (data["costs"] / 10000).astype(int)
    packageForFitness = pd.DataFrame({"lengths":lengths, "realCosts": realCosts, "freqs": data.frequencies})         
    
    return x, y, number_of_activities, max_len, index_to_word, packageForFitness, minLengthRun, m_AC   

In [5]:
# example of use
x, y, number_of_activities, max_len, index_to_word, packageForFitness, minLengthRun, m_AC  = cleanDataForRNN("alignments-A_2012_im.csv",5)
x, y

(array([[1., 2., 3., ..., 0., 0., 0.],
        [1., 2., 4., ..., 0., 0., 0.],
        [1., 2., 4., ..., 0., 0., 0.],
        ...,
        [1., 2., 5., ..., 0., 0., 0.],
        [1., 2., 5., ..., 0., 0., 0.],
        [1., 2., 4., ..., 0., 0., 0.]]),
 array([[0., 1.],
        [0., 1.],
        [0., 1.],
        ...,
        [0., 1.],
        [0., 1.],
        [0., 1.]]))

# 2. Model

In [6]:
def myModel(max_len, number_of_activities, verbose=True):
    """    
    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    size_of_voc -- size of the embedding output
    
    Returns:
    model -- a model instance in Keras
    """
    # Input of the model, type is dtype 'int32' (as it contains indices of activities, which are integers).
    indices = Input(shape= (max_len,), dtype='int32')
    # Create the embedding layer to reduce to voc size
    embeddings = Embedding(input_dim= number_of_activities, output_dim= 15, input_length=max_len)(indices)
    # Propagate the embeddings through an Bi-LSTM layer with 50-dimensional hidden state 
    # Return the batch of sequences (to replay)
    X = Bidirectional(LSTM(units = 50, return_sequences=True, go_backwards=True),merge_mode='concat')(embeddings)
    # Add dropout with a probability of 0.5 to help with overfitting
    X = Dropout(0.5)(X)
    # Propagate X trough another LSTM layer with 50-dimensional hidden state
    X = LSTM(units = 50, return_sequences=False)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(0.5)(X)
    # Propagate X through a Dense layer of the number of predicting classes
    X = Dense(units=2)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)
    
    # Create Model instance which converts indices into X.
    model = Model(inputs=indices, outputs=X)
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    if verbose:
        print(model.summary())
        
    return model

In [7]:
# example of use
model = myModel(max_len, number_of_activities)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 176)]             0         
                                                                 
 tf.compat.v1.nn.embedding_l  (None, 176, 15)          0         
 ookup (TFOpLambda)                                              
                                                                 
 bidirectional (Bidirectiona  (None, 176, 100)         26400     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 176, 100)          0         
                                                                 
 lstm_1 (LSTM)               (None, 50)                30200     
                                                                 
 dropout_1 (Dropout)         (None, 50)                0     

# 3. Cross-Validation of the method

In [8]:
def accLossPercentageForRNN(model, x, y, indices_of_test, accArr, lossArr, percentageArr=None,  classToTest=None, verbose=2,freqs=None):
    '''
    This function fills the arrays of results accArr, lossArr and percentageArr for all the test items, or only the 
    negative items (classToTest=0) or only the positive items (classToTest=1). percentageArr is optional because it 
    is not required for the entire dataset, i.e., when we do not specify the class. 
    This works by using a dynamic programmation for the arrays. The return element is either the current accuracy in 
    case of classToTest=None, either the indices in case of classToTest!=None.
    Params:
    @forest: a trained model
    @x_test: the dataset to test
    @y_test: the target value to predict for the test dataset
    @accArr: a list of the previous accurary, or an empty list
    @lossArr: a list of the previous loss, or an empty list
    @percentageArr:  a list of the previous percentage, or an empty list
    @classToTest: 1 or 0 for positive and negative. This will find the indices of the items that belongs to the class
    '''
    if classToTest!=None:
        indices = [i for i in indices_of_test if y[i][1]==classToTest]
        if len(indices)>0:
            loss, acc = model.evaluate(x[indices], y[indices],verbose=verbose)
            accArr.append(acc)
            lossArr.append(loss)
        #percentageArr.append(freqs[indices].sum()/freqs[indices_of_test].sum())
        percentageArr.append(len(indices)/len(indices_of_test))
        return indices
    else :
        loss, acc = model.evaluate(x[indices_of_test], y[indices_of_test],verbose=verbose)
        accArr.append(acc)
        lossArr.append(loss)
        return acc

In [10]:
def runKFoldForRNN(numberOfFold, x, y, packageForFitness,minLengthRun, m_AC,
                   max_len, number_of_activities, verbose=2,epochs = 10, batch_size = 50):
    '''
    This function runs a RNN and prints some metrics. 
    
    @model: the created RNN
    @numberOfFold: number of fold for the cross validation
    @x: sequences
    @y: target
    '''    
    startModel = time.time()
    accAll, lossAll = [], []
    accNeg, lossNeg, percentageNeg = [], [], []
    accPos, lossPos, percentagePos = [], [], []
    
    realFitness, realLBFitness, predictedLBFitness = [], [], []
    packageForFitness.reset_index(drop=True,inplace=True)
    
    runtime = []

    # use a K-fold Cross-Validation and show average Loss and average Accuracy
    kfold = KFold(numberOfFold)
    bestmodel, bestAccuracy = None, 0
    
    for indices_of_train, indices_of_test in kfold.split(x):
        
        # train the model
        model = myModel(max_len, number_of_activities+1,verbose)
        model.fit(x[indices_of_train], y[indices_of_train], verbose=verbose, epochs = 10, batch_size = 50)
        
        # compute loss and accuracy on the test items
        current_accuracy = accLossPercentageForRNN(model, x, y, indices_of_test, accAll, lossAll, verbose=verbose)
        
        # compute loss and accuracy on the test items that are negatives
        accLossPercentageForRNN(model, x, y, indices_of_test, accNeg, lossNeg, percentageNeg, 0, verbose)

        # compute loss and accuracy on the test items that are positives
        indices_of_pos = accLossPercentageForRNN(model, x, y, indices_of_test, accPos, lossPos, percentagePos, 1, verbose) 

        # compute fitness and lower-bound
        realFitness.append(fitness(packageForFitness.iloc[indices_of_test], minLengthRun))
        if len(indices_of_pos)>0:
            realLBFitness.append(LB_fitness(packageForFitness.iloc[indices_of_test], minLengthRun, m_AC, indices_of_pos))
        
        # compute predicted LB fitness and runtime
        start = time.time()
        predictions = model.predict(x[indices_of_test])
        runtime.append((time.time()-start)/len(indices_of_test))
        indices_of_predicted_as_positives = [indices_of_test[i] for i in range(0,len(predictions)) if predictions[i][1]>=0.5]
        if len(indices_of_predicted_as_positives)>0:
            predictedLBFitness.append(LB_fitness(packageForFitness.iloc[indices_of_test], minLengthRun, m_AC, indices_of_predicted_as_positives))

        if bestAccuracy < current_accuracy:
            bestmodel = model
            bestAccuracy = current_accuracy
    print("TIME:",time.time()-startModel)   
    print("[CROSS-VALIDATION]\n[ALL] Loss:", "{:.3f}".format(mean(lossAll)), "\t Acc:", "{:.3f}".format(mean(accAll)))
    print("[POSITIVE ({:.2f}%)] Loss:".format(mean(percentagePos)), "{:.3f}".format(mean(lossPos)), "\t Acc:", "{:.3f}".format(mean(accPos)))
    print("[NEGATIVE ({:.2f}%)] Loss:".format(mean(percentageNeg)), "{:.3f}".format(mean(lossNeg)),"\t Acc:", "{:.3f}".format(mean(accNeg)))
    print("Fitness {:.3f}".format(mean(realFitness)), "\t LB Fitness:", "{:.3f}\n".format(mean(realLBFitness)),"\t Predicted LB Fitness:", "{:.3f}\n".format(mean(predictedLBFitness)))
    print("Runtime (prediction per trace):{:.10f}".format(mean(runtime)))
    return bestmodel

In [11]:
# example of use
runKFoldForRNN(3,x,y, packageForFitness,minLengthRun, m_AC,max_len, number_of_activities)

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 176)]             0         
                                                                 
 tf.compat.v1.nn.embedding_l  (None, 176, 15)          0         
 ookup_1 (TFOpLambda)                                            
                                                                 
 bidirectional_1 (Bidirectio  (None, 176, 100)         26400     
 nal)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 176, 100)          0         
                                                                 
 lstm_3 (LSTM)               (None, 50)                30200     
                                                                 
 dropout_3 (Dropout)         (None, 50)                0   

Epoch 4/10
59/59 - 13s - loss: 0.0073 - accuracy: 0.9993 - 13s/epoch - 226ms/step
Epoch 5/10
59/59 - 13s - loss: 0.0074 - accuracy: 0.9993 - 13s/epoch - 226ms/step
Epoch 6/10
59/59 - 14s - loss: 0.0067 - accuracy: 0.9993 - 14s/epoch - 244ms/step
Epoch 7/10
59/59 - 14s - loss: 0.0068 - accuracy: 0.9993 - 14s/epoch - 242ms/step
Epoch 8/10
59/59 - 16s - loss: 0.0069 - accuracy: 0.9993 - 16s/epoch - 273ms/step
Epoch 9/10
59/59 - 15s - loss: 0.0058 - accuracy: 0.9993 - 15s/epoch - 261ms/step
Epoch 10/10
59/59 - 15s - loss: 0.0059 - accuracy: 0.9993 - 15s/epoch - 248ms/step
46/46 - 6s - loss: 0.0058 - accuracy: 0.9993 - 6s/epoch - 120ms/step
1/1 - 0s - loss: 8.0340 - accuracy: 0.0000e+00 - 43ms/epoch - 43ms/step
46/46 - 3s - loss: 3.2493e-04 - accuracy: 1.0000 - 3s/epoch - 56ms/step
46/46 [==============================] - 5s 64ms/step
TIME: 497.9206864833832
[CROSS-VALIDATION]
[ALL] Loss: 0.006 	 Acc: 0.999
[POSITIVE (1.00%)] Loss: 0.000 	 Acc: 1.000
[NEGATIVE (0.00%)] Loss: 8.102 	 Acc: 0.

# 4. Train and Test

In [12]:
filename = "A_2017_im.csv"

for i in [2,4,6,8,10]:
    print("\n","m_AC=",i)
    
    # read the datafile, packageForFitness, minLengthRun, m_AC are needed to compute fitness and LBfitness
    x, y, number_of_activities, max_len, index_to_word, packageForFitness, minLengthRun, m_AC  = cleanDataForRNN("alignments-"+filename,i)
    
    # split the dataset in TRAIN and TEST sets
    X_train, X_test, y_train, y_test, packageForFitness_train, packageForFitness_test = train_test_split(x, y, packageForFitness, test_size=0.33, random_state=42)

    # ----------------------------------------------
    #                     TRAIN 
    # ----------------------------------------------
    # run the cross-validation
    bestmodel = runKFoldForRNN(10, X_train, y_train, packageForFitness_train, minLengthRun, m_AC, max_len, number_of_activities, verbose=0)

    # ----------------------------------------------
    #                     TEST 
    # ----------------------------------------------
    # use the same function as in the cross-validation but for the test set. 
    accAll, lossAll = [], []
    accNeg, lossNeg, percentageNeg = [], [], []
    accPos, lossPos, percentagePos = [], [], []
    
    packageForFitness_test.reset_index(drop=True,inplace=True)
    
    #compute loss and accuracy on the test items 
    accLossPercentageForRNN(bestmodel, X_test, y_test, range(0,len(y_test)), accAll, lossAll, verbose=0)
        
    # compute loss and accuracy on the test items that are negatives
    accLossPercentageForRNN(bestmodel, X_test, y_test, range(0,len(y_test)), accNeg, lossNeg, percentageNeg, 0, 0)

    # compute loss and accuracy on the test items that are positives
    indices_of_pos = accLossPercentageForRNN(bestmodel, X_test, y_test, range(0,len(y_test)), accPos, lossPos,percentagePos,  1, 0) 

    # compute fitness and lower-bound
    realFitness = fitness(packageForFitness_test, minLengthRun)
    if len(indices_of_pos)>0:
        LBFitness = LB_fitness(packageForFitness_test, minLengthRun, m_AC, indices_of_pos)
        
    # compute predicted LB fitness
    predictions = bestmodel.predict(X_test)
    indices_of_predicted_as_positives = [i for i in range(0, len(X_test)) if predictions[i][1]>=0.5]
    if len(indices_of_predicted_as_positives)>0:
        predictedLBFitness = LB_fitness(packageForFitness_test, minLengthRun, m_AC, indices_of_predicted_as_positives)
        
    print("[TEST]\n[ALL] Loss:", "{:.3f}".format(mean(lossAll)), "\t Acc:", "{:.3f}".format(mean(accAll)))
    print("[POSITIVE ({:.2f}%)] Loss:".format(mean(percentagePos)), "{:.3f}".format(mean(lossPos)), "\t Acc:", "{:.3f}".format(mean(accPos)))
    print("[NEGATIVE ({:.2f}%)] Loss:".format(mean(percentageNeg)), "{:.3f}".format(mean(lossNeg)),"\t Acc:", "{:.3f}".format(mean(accNeg)))
    print("Fitness {:.3f}".format((realFitness)), "\t LB Fitness:", "{:.3f}\n".format((LBFitness)),"\t Predicted LB Fitness:", "{:.3f}\n".format((predictedLBFitness)) )

    fake_x, fake_y, fake_number_of_activities, fake_max_len, fake_index_to_word, fake_packageForFitness, fake_minLengthRun, fake_m_AC  = cleanDataForRNN("alignments-mock-"+filename,2,word_to_index_already_exists=True)
    accFake, lossFake = [], []
    accLossPercentageForRNN(bestmodel, fake_x, fake_y, range(0,len(fake_y)), accFake, lossFake)
    print("[MOCK] Loss:", "{:.3f}".format(mean(lossFake)), "\t Acc:", "{:.3f}".format(mean(accFake)))
    print("-------------------------------------------")


 m_AC= 2
34/34 [==============================] - 4s 73ms/step
TIME: 5054.241726875305
[CROSS-VALIDATION]
[ALL] Loss: 0.255 	 Acc: 0.932
[POSITIVE (0.07%)] Loss: 2.634 	 Acc: 0.100
[NEGATIVE (0.93%)] Loss: 0.074 	 Acc: 0.998
Fitness 0.873 	 LB Fitness: 0.073
 	 Predicted LB Fitness: 0.092

Runtime (prediction per trace):0.0032249906
165/165 [==============================] - 13s 81ms/step
[TEST]
[ALL] Loss: 0.042 	 Acc: 0.984
[POSITIVE (0.07%)] Loss: 0.210 	 Acc: 0.972
[NEGATIVE (0.93%)] Loss: 0.029 	 Acc: 0.985
Fitness 0.874 	 LB Fitness: 0.073
 	 Predicted LB Fitness: 0.085

32/32 - 1s - loss: 0.0676 - accuracy: 0.9600 - 1s/epoch - 46ms/step
[MOCK] Loss: 0.068 	 Acc: 0.960
-------------------------------------------

 m_AC= 4
34/34 [==============================] - 3s 56ms/step


34/34 [==============================] - 3s 61ms/step
TIME: 4625.849546670914
[CROSS-VALIDATION]
[ALL] Loss: 0.288 	 Acc: 0.897
[POSITIVE (0.12%)] Loss: 1.707 	 Acc: 0.174
[NEGATIVE (0.88%)] Loss: 0.094 	 Acc: 0.996
Fitness 0.873 	 LB Fitness: 0.114
 	 Predicted LB Fitness: 0.075

Runtime (prediction per trace):0.0030364123
165/165 [==============================] - 10s 58ms/step
[TEST]
[ALL] Loss: 0.064 	 Acc: 0.973
[POSITIVE (0.12%)] Loss: 0.172 	 Acc: 0.926
[NEGATIVE (0.88%)] Loss: 0.049 	 Acc: 0.980
Fitness 0.874 	 LB Fitness: 0.115
 	 Predicted LB Fitness: 0.124

32/32 - 2s - loss: 0.2797 - accuracy: 0.8959 - 2s/epoch - 56ms/step
[MOCK] Loss: 0.280 	 Acc: 0.896
-------------------------------------------

 m_AC= 6
34/34 [==============================] - 4s 60ms/step
TIME: 4763.572948217392
[CROSS-VALIDATION]
[ALL] Loss: 0.205 	 Acc: 0.920
[POSITIVE (0.18%)] Loss: 0.710 	 Acc: 0.621
[NEGATIVE (0.82%)] Loss: 0.091 	 Acc: 0.988
Fitness 0.873 	 LB Fitness: 0.171
 	 Predicted LB Fitne

34/34 [==============================] - 4s 80ms/step
TIME: 6270.219386577606
[CROSS-VALIDATION]
[ALL] Loss: 0.166 	 Acc: 0.927
[POSITIVE (0.29%)] Loss: 0.256 	 Acc: 0.856
[NEGATIVE (0.71%)] Loss: 0.129 	 Acc: 0.955
Fitness 0.873 	 LB Fitness: 0.270
 	 Predicted LB Fitness: 0.291

Runtime (prediction per trace):0.0038394067
165/165 [==============================] - 13s 80ms/step
[TEST]
[ALL] Loss: 0.101 	 Acc: 0.958
[POSITIVE (0.30%)] Loss: 0.162 	 Acc: 0.944
[NEGATIVE (0.70%)] Loss: 0.074 	 Acc: 0.964
Fitness 0.874 	 LB Fitness: 0.276
 	 Predicted LB Fitness: 0.284

32/32 - 2s - loss: 0.2913 - accuracy: 0.9179 - 2s/epoch - 76ms/step
[MOCK] Loss: 0.291 	 Acc: 0.918
-------------------------------------------

 m_AC= 10
34/34 [==============================] - 6s 158ms/step
TIME: 9890.342503070831
[CROSS-VALIDATION]
[ALL] Loss: 0.225 	 Acc: 0.885
[POSITIVE (0.39%)] Loss: 0.283 	 Acc: 0.765
[NEGATIVE (0.61%)] Loss: 0.190 	 Acc: 0.961
Fitness 0.873 	 LB Fitness: 0.354
 	 Predicted LB Fit